révision: 26/09/2024

# TP 2024 Fenêtres OVER,      
Sujet seul sans solutions

*   **Données**: Foursquare
*   **Moteur SQL**: DuckDB


FERCHE Adelin-Flaviu

3800655

Master 2 SAR




# Préparation

Lire la [documentation DuckDB SQL](https://duckdb.org/docs/sql/introduction.html) :
  * [Functions](https://duckdb.org/docs/sql/functions/overview)
  * [Window functions](https://duckdb.org/docs/sql/window_functions)


Affichage convivial des résultats d'une requête

In [1]:
pip install -q itables

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.0 MB/s eta 0:00:00


In [2]:
# pour améliorer l'affichage des tableaux contenant le resultat d'une requete
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

In [3]:
import pandas as pd

def display(query, n=30):
    return query.limit(n).df()

# seulement dans colab
# from google.colab import data_table
# def display(query, n=100):
#     return data_table.DataTable(query.limit(n).df(), include_index=False, num_rows_per_page=10)


In [4]:
!pip install duckdb

Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [5]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + "\n"
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    derniere = None
    est_requete = False
    for r in tabRequetes:
        r = r.strip()
        if len(r) > 2:
          derniere = db.sql(r)
          est_requete = ( r.lower().startswith('select')or r.lower().startswith('with'))
    if(est_requete):
      return display(derniere)
    else:
      return print('ok')

print("fonction définie")

fonction définie


# Démarrer le service DuckDB

In [6]:
import duckdb

# db = duckdb.connect('foursquare')
db = duckdb.connect(':memory:')

# vérifier que le service fonctrionne
r = db.sql("SELECT 'hello' as col")
display(r)

col
Loading ITables v2.2.2 from the internet... (need help?)


In [7]:
def display_schema(table):
  return display(db.sql(f"""
    SELECT column_name, data_type
    FROM duckdb_columns
    WHERE table_name = '{table}'
  """))

def display_all_views():
  return display(db.sql("select view_name, sql, temporary from duckdb_views"))

print("fonctions définies")

fonctions définies


# Premier exemple

Un premier exemple pour comprendre les fonctions sur des fenêtres

## Les données

### Les visites

In [8]:
visite_attrs = ["photoId",	"personId",	"date",	"lon",	"lat", "note"]
visite_tuples = [("p1",	"Bob",	"03/09/2020",	41.5,	12.8, 3),
                 ("p2",	"Alice", "01/09/2020",	41.6,	12.8, 5),
                 ("p3",	"Bob",	"04/09/2020",	41.6,	13.2, 1),
                 ("p4",	"Bob",	"04/09/2020",	40.1,	12.0, 2),
                 ("p5",	"Alice", "04/09/2020",	40.1,	12.0, 2),
                 ("p6",	"Alice", "05/09/2020",	41.6,	12.7, 4),
                 ("p7",	"Alice", "05/10/2020",	41.8, 12.8, 4),
                 ("p8",	"Carole", "25/12/2019",	30.1, 10.1, 2),
                 ("p9",	"David", "25/12/2019",	30.1, 10.1, 1),
                 ("p10","Eva", "25/12/2019",	30.1, 10.1, 5),
                 ("p11","Eva", "26/12/2019",	31.1, 10.1, 3),
                 ("p12","Alice", "01/02/2020",	32.1, 12.1, 3),
                 ("p13","Bob", "01/02/2020",	32.1, 12.1, 5),
                 ("p14","Carole", "01/02/2020",	32.1, 12.1, 2),
                 ("p15",	"Carole", "11/11/2019",	49.1, 10.1, 1),
                 ("p16",	"Alice", "25/12/2019",	30.1, 10.1, 4),
                 ("p17",	"Alice", "02/02/2020",	49.1, 10.1, 5),
                ]

visites_df = pd.DataFrame(visite_tuples, columns=visite_attrs)

db.sql("""
DROP TABLE IF EXISTS Visites CASCADE;

CREATE TABLE Visites AS
SELECT photoId,
       personId,
       strptime(date, '%d/%m/%Y') as date_num,
       extract(year from date_num) as année,
       extract(month from date_num) as mois,
       lon,
      lat,
      note
FROM visites_df""")

Visites = db.table("Visites")

display(Visites)

photoId personId   date_num  année  mois   lon   lat  note
0       p1      Bob 2020-09-03   2020     9  41.5  12.8     3
1       p2    Alice 2020-09-01   2020     9  41.6  12.8     5
2       p3      Bob 2020-09-04   2020     9  41.6  13.2     1
3       p4      Bob 2020-09-04   2020     9  40.1  12.0     2
4       p5    Alice 2020-09-04   2020     9  40.1  12.0     2
5       p6    Alice 2020-09-05   2020     9  41.6  12.7     4
6       p7    Alice 2020-10-05   2020    10  41.8  12.8     4
7       p8   Carole 2019-12-25   2019    12  30.1  10.1     2
8       p9    David 2019-12-25   2019    12  30.1  10.1     1
9      p10      Eva 2019-12-25   2019    12  30.1  10.1     5
10     p11      Eva 2019-12-26   2019    12  31.1  10.1     3
11     p12    Alice 2020-02-01   2020     2  32.1  12.1     3
12     p13      Bob 2020-02-01   2020     2  32.1  12.1     5
13     p14   Carole 2020-02-01   2020     2  32.1  12.1     2
14     p15   Carole 2019-11-11   2019    11  49.1  10.1     1
15     p16    Alice 2019-12-25   2019    12  30.1  10.1     4
16     p17    Alice 2020-02-02   2020     2  49.1  10.1     5

In [9]:
display_schema("Visites")

column_name  data_type
0     photoId    VARCHAR
1    personId    VARCHAR
2    date_num  TIMESTAMP
3       année     BIGINT
4        mois     BIGINT
5         lon     DOUBLE
6         lat     DOUBLE
7        note     BIGINT

In [10]:
# # exemple de cellule avec requete SQL

%%sql
SELECT *
FROM Visites

photoId personId   date_num  année  mois   lon   lat  note
0       p1      Bob 2020-09-03   2020     9  41.5  12.8     3
1       p2    Alice 2020-09-01   2020     9  41.6  12.8     5
2       p3      Bob 2020-09-04   2020     9  41.6  13.2     1
3       p4      Bob 2020-09-04   2020     9  40.1  12.0     2
4       p5    Alice 2020-09-04   2020     9  40.1  12.0     2
5       p6    Alice 2020-09-05   2020     9  41.6  12.7     4
6       p7    Alice 2020-10-05   2020    10  41.8  12.8     4
7       p8   Carole 2019-12-25   2019    12  30.1  10.1     2
8       p9    David 2019-12-25   2019    12  30.1  10.1     1
9      p10      Eva 2019-12-25   2019    12  30.1  10.1     5
10     p11      Eva 2019-12-26   2019    12  31.1  10.1     3
11     p12    Alice 2020-02-01   2020     2  32.1  12.1     3
12     p13      Bob 2020-02-01   2020     2  32.1  12.1     5
13     p14   Carole 2020-02-01   2020     2  32.1  12.1     2
14     p15   Carole 2019-11-11   2019    11  49.1  10.1     1
15     p16    Alice 2019-12-25   2019    12  30.1  10.1     4
16     p17    Alice 2020-02-02   2020     2  49.1  10.1     5

###Les lieux

In [11]:
lieux_attributs = ["lon","lat", "ville", "pays"]

lieux_tuples = [(41.5, 12.8, "Aix", "France"),
                (41.6, 12.8, "Aix", "France"),
                (41.6, 12.7, "Nice", "France"),
                (41.8, 12.8, "Marseille", "France"),
                (41.6, 13.2, "Rome", "Italie"),
                (40.1, 12.0, "Oslo", "Norvege"),
                (30.1, 10.1, "Paris", "France"),
                (31.1, 10.1, "StDenis", "France"),
                (32.1, 12.1, "Lille", "France"),
                (49.1, 10.1, "Pau", "France") ]


lieux_df = pd.DataFrame(lieux_tuples, columns=lieux_attributs)
db.sql("""
drop table if exists Lieux;
create table Lieux as
select ville, pays, lon, lat from lieux_df""")

Lieux = db.table("Lieux")

display(Lieux)

ville     pays   lon   lat
0        Aix   France  41.5  12.8
1        Aix   France  41.6  12.8
2       Nice   France  41.6  12.7
3  Marseille   France  41.8  12.8
4       Rome   Italie  41.6  13.2
5       Oslo  Norvege  40.1  12.0
6      Paris   France  30.1  10.1
7    StDenis   France  31.1  10.1
8      Lille   France  32.1  12.1
9        Pau   France  49.1  10.1

### Les professions

In [12]:
profession_attributs = ["personID","profession"]

profession_tuples = [("Alice", "Architecte"),
                     ("Bob", "Data science"),
                     ("Carole", "Data science"),
                     ("David", "Architecte"),
                     ("Eva", "Vendeuse"),
                     ("Franck", "Vendeur"),
                     ("Greta", "Economiste")
                    ]

profession_df = pd.DataFrame(profession_tuples, columns=profession_attributs)
db.sql("""
drop table if exists Professions;
create table Professions as
select * from profession_df""")

Professions = db.table("Professions")

display(Professions)

personID    profession
0    Alice    Architecte
1      Bob  Data science
2   Carole  Data science
3    David    Architecte
4      Eva      Vendeuse
5   Franck       Vendeur
6    Greta    Economiste

### Le détail des visites

Définir une vue contenant le détail des visites

In [13]:
db.sql("""
CREATE OR REPLACE VIEW VisitesDetail AS

SELECT v.photoID, p.personID, p.profession, v.date_num, v.mois, v.année, v.lon, v.lat, l.ville, l.pays,  v.note
FROM Visites v
JOIN Lieux l ON (v.lon = l.lon AND v.lat = l.lat )
JOIN Professions p ON (v.personId = p.personId)
ORDER BY pays, ville
""")

# Même requête mais écrite avec un NATURAL JOIN
# visitesDetail = db.sql("""
# CREATE OR REPLACE VIEW VisitesDetail AS
# SELECT v.photoID, p.personID, p.profession, v.date_num, v.mois, v.année, v.lon, v.lat, l.ville, l.pays,  v.note
# FROM Visites v NATURAL JOIN Lieux l NATURAL JOIN Professions p
# ORDER BY pays, ville
# """)

VisitesDetail=db.view('VisitesDetail')

display(VisitesDetail)

photoId personID    profession   date_num  mois  année   lon   lat  \
0       p1      Bob  Data science 2020-09-03     9   2020  41.5  12.8   
1       p2    Alice    Architecte 2020-09-01     9   2020  41.6  12.8   
2      p12    Alice    Architecte 2020-02-01     2   2020  32.1  12.1   
3      p13      Bob  Data science 2020-02-01     2   2020  32.1  12.1   
4      p14   Carole  Data science 2020-02-01     2   2020  32.1  12.1   
5       p7    Alice    Architecte 2020-10-05    10   2020  41.8  12.8   
6       p6    Alice    Architecte 2020-09-05     9   2020  41.6  12.7   
7       p8   Carole  Data science 2019-12-25    12   2019  30.1  10.1   
8       p9    David    Architecte 2019-12-25    12   2019  30.1  10.1   
9      p10      Eva      Vendeuse 2019-12-25    12   2019  30.1  10.1   
10     p16    Alice    Architecte 2019-12-25    12   2019  30.1  10.1   
11     p15   Carole  Data science 2019-11-11    11   2019  49.1  10.1   
12     p17    Alice    Architecte 2020-02-02     2   2020  49.1  10.1   
13     p11      Eva      Vendeuse 2019-12-26    12   2019  31.1  10.1   
14      p3      Bob  Data science 2020-09-04     9   2020  41.6  13.2   
15      p4      Bob  Data science 2020-09-04     9   2020  40.1  12.0   
16      p5    Alice    Architecte 2020-09-04     9   2020  40.1  12.0   

        ville     pays  note  
0         Aix   France     3  
1         Aix   France     5  
2       Lille   France     3  
3       Lille   France     5  
4       Lille   France     2  
5   Marseille   France     4  
6        Nice   France     4  
7       Paris   France     2  
8       Paris   France     1  
9       Paris   France     5  
10      Paris   France     4  
11        Pau   France     1  
12        Pau   France     5  
13    StDenis   France     3  
14       Rome   Italie     1  
15       Oslo  Norvege     2  
16       Oslo  Norvege     2

## Requete avec fenêtre de taille croissante

Numéroter les tuples

In [14]:
r = db.sql("""
SELECT v.personId, v.photoId, v.note,
       row_number() OVER (ORDER BY personId, photoId) as numeroPhoto
FROM Visites v
""")

display(r)

personId photoId  note  numeroPhoto
0     Alice     p12     3            1
1     Alice     p16     4            2
2     Alice     p17     5            3
3     Alice      p2     5            4
4     Alice      p5     2            5
5     Alice      p6     4            6
6     Alice      p7     4            7
7       Bob      p1     3            8
8       Bob     p13     5            9
9       Bob      p3     1           10
10      Bob      p4     2           11
11   Carole     p14     2           12
12   Carole     p15     1           13
13   Carole      p8     2           14
14    David      p9     1           15
15      Eva     p10     5           16
16      Eva     p11     3           17

Classement des visites par note décroissantes

In [15]:
r = db.sql("""
SELECT v.personId, v.photoId, v.note, rank() over (order by note desc) as rang
FROM Visites v
""")
display(r)

personId photoId  note  rang
0     Alice      p2     5     1
1       Eva     p10     5     1
2       Bob     p13     5     1
3     Alice     p17     5     1
4     Alice      p6     4     5
5     Alice      p7     4     5
6     Alice     p16     4     5
7       Bob      p1     3     8
8       Eva     p11     3     8
9     Alice     p12     3     8
10      Bob      p4     2    11
11    Alice      p5     2    11
12   Carole      p8     2    11
13   Carole     p14     2    11
14      Bob      p3     1    15
15    David      p9     1    15
16   Carole     p15     1    15

## Requête Top K
 Le top 5 des Visites avec les meilleures notes.
Il peut y avoir plus de k visites dans le résultat à cause des **ex aequos**.

In [16]:
r=db.sql("""
WITH Classement as (
  SELECT v.personId,
         v.photoId, v.note,
         rank() over (order by note desc) as rang
  FROM Visites v
)

SELECT *
FROM Classement
WHERE rang <=5
ORDER BY rang
""")

display(r)

personId photoId  note  rang
0    Alice      p2     5     1
1      Eva     p10     5     1
2      Bob     p13     5     1
3    Alice     p17     5     1
4    Alice      p6     4     5
5    Alice      p7     4     5
6    Alice     p16     4     5

Classement dense

In [17]:
r=db.sql("""
SELECT  v.personId,
        v.photoId,
        v.note,
        dense_rank() over (order by note desc) as rang_dense
FROM Visites v
""")

display(r)

personId photoId  note  rang_dense
0     Alice      p2     5           1
1       Eva     p10     5           1
2       Bob     p13     5           1
3     Alice     p17     5           1
4     Alice      p6     4           2
5     Alice      p7     4           2
6     Alice     p16     4           2
7       Bob      p1     3           3
8       Eva     p11     3           3
9     Alice     p12     3           3
10      Bob      p4     2           4
11    Alice      p5     2           4
12   Carole      p8     2           4
13   Carole     p14     2           4
14      Bob      p3     1           5
15    David      p9     1           5
16   Carole     p15     1           5

## Fenêtrage et partitionnement : Partition by


Une partition est définie par un sous-ensemble des attributs projetés dans le select.

Une fenêtre par partition.

Le classement **par ville** des visites les mieux notées

In [18]:
%%sql

SELECT v.photoId, v.personID, v.ville,
       rank() OVER( PARTITION BY ville ORDER BY note) as rang

FROM VisitesDetail v

ORDER BY ville, rang

photoId personID      ville  rang
0       p1      Bob        Aix     1
1       p2    Alice        Aix     2
2      p14   Carole      Lille     1
3      p12    Alice      Lille     2
4      p13      Bob      Lille     3
5       p7    Alice  Marseille     1
6       p6    Alice       Nice     1
7       p4      Bob       Oslo     1
8       p5    Alice       Oslo     1
9       p9    David      Paris     1
10      p8   Carole      Paris     2
11     p16    Alice      Paris     3
12     p10      Eva      Paris     4
13     p15   Carole        Pau     1
14     p17    Alice        Pau     2
15      p3      Bob       Rome     1
16     p11      Eva    StDenis     1

Le classement **par pays**  des villes ayant de plus de visites

In [19]:
%%sql
SELECT pays, ville, count(*) as nbVisite,
       rank() over (partition by pays order by count(*) desc) as rang
From VisitesDetail v
Group by pays, ville
Order by pays, rang

pays      ville  nbVisite  rang
0   France      Paris         4     1
1   France      Lille         3     2
2   France        Aix         2     3
3   France        Pau         2     3
4   France  Marseille         1     5
5   France    StDenis         1     5
6   France       Nice         1     5
7   Italie       Rome         1     1
8  Norvege       Oslo         2     1

## Questions

### Question 1 : temps écoulé

Pour chaque visite de chaque personne, le temps écoulé depuis la **première** visite de cette personne. Exprimer le temps écoulé par un intervalle de temps résultant d'une différence entre deux dates. L'intervalle indique le nombre de jours.

Le schema attendu est (photoID , personID, date, datePremiereVisiste, jours_ecoules)

Astuce: commencer par compléter chaque visite avec la date de la première visite de la personne.

In [20]:
%%sql
with T as (
  select  photoID , personID, date_num,
   min(date_num) over (partition by personId) as minD
  from Visites
)

SELECT photoID, personID, date_num, minD, date_num-minD as jours_ecoules
FROM T
ORDER BY personID, date_num

photoId personId   date_num       minD jours_ecoules
0      p16    Alice 2019-12-25 2019-12-25        0 days
1      p12    Alice 2020-02-01 2019-12-25       38 days
2      p17    Alice 2020-02-02 2019-12-25       39 days
3       p2    Alice 2020-09-01 2019-12-25      251 days
4       p5    Alice 2020-09-04 2019-12-25      254 days
5       p6    Alice 2020-09-05 2019-12-25      255 days
6       p7    Alice 2020-10-05 2019-12-25      285 days
7      p13      Bob 2020-02-01 2020-02-01        0 days
8       p1      Bob 2020-09-03 2020-02-01      215 days
9       p3      Bob 2020-09-04 2020-02-01      216 days
10      p4      Bob 2020-09-04 2020-02-01      216 days
11     p15   Carole 2019-11-11 2019-11-11        0 days
12      p8   Carole 2019-12-25 2019-11-11       44 days
13     p14   Carole 2020-02-01 2019-11-11       82 days
14      p9    David 2019-12-25 2019-12-25        0 days
15     p10      Eva 2019-12-25 2019-12-25        0 days
16     p11      Eva 2019-12-26 2019-12-25        1 days

### Question 2: Requête avec fenêtre glissante de taille fixe

Pour chaque visite de chaque personne, le nombre de jours depuis la visite **précédente** de cette personne.

Le schéma attendu est (photoID, personID, date, precedenteDate, nbJours)

Astuce: penser à utiliser la fonction datediff pour comparer deux dates et obtenir un nombre de jours


In [21]:
%%sql
WITH
V2 as (
  select  photoID , personID, date_num,
  min(date_num) over (
    partition by personId
    order by date_num
    rows between 1 preceding and 1 following
  ) as precedenteDate
 from Visites
)

select photoID, personID, date_num, precedenteDate, datediff('day', precedenteDate, date_num) as nbJours
from V2
order by personID, date_num

photoId personId   date_num precedenteDate  nbJours
0      p16    Alice 2019-12-25     2019-12-25        0
1      p12    Alice 2020-02-01     2019-12-25       38
2      p17    Alice 2020-02-02     2020-02-01        1
3       p2    Alice 2020-09-01     2020-02-02      212
4       p5    Alice 2020-09-04     2020-09-01        3
5       p6    Alice 2020-09-05     2020-09-04        1
6       p7    Alice 2020-10-05     2020-09-05       30
7      p13      Bob 2020-02-01     2020-02-01        0
8       p1      Bob 2020-09-03     2020-02-01      215
9       p3      Bob 2020-09-04     2020-09-03        1
10      p4      Bob 2020-09-04     2020-09-04        0
11     p15   Carole 2019-11-11     2019-11-11        0
12      p8   Carole 2019-12-25     2019-11-11       44
13     p14   Carole 2020-02-01     2019-12-25       38
14      p9    David 2019-12-25     2019-12-25        0
15     p10      Eva 2019-12-25     2019-12-25        0
16     p11      Eva 2019-12-26     2019-12-25        1


## Question 3 : Semaine glissante
Pour chaque jour de visite de chaque personne, le nombre de visite sur 7 jours glissants.

Schéma: (personId, date_num, nbVisites_depuis_7jours)

In [22]:
%%sql
WITH T as (
  SELECT  personId, date_num, count(*) as nb_visites
  FROM Visites
  GROUP BY personId, date_num
  ORDER BY personId, date_num
)

SELECT  personID, date_num,
    sum(nb_visites) over(
    partition by personId
    order by date_num
    range between interval 7 day preceding and interval 0 day following
  ) as nbVisites_depuis_7jours
FROM T
ORDER BY personID, date_num

personId   date_num  nbVisites_depuis_7jours
0     Alice 2019-12-25                      1.0
1     Alice 2020-02-01                      1.0
2     Alice 2020-02-02                      2.0
3     Alice 2020-09-01                      1.0
4     Alice 2020-09-04                      2.0
5     Alice 2020-09-05                      3.0
6     Alice 2020-10-05                      1.0
7       Bob 2020-02-01                      1.0
8       Bob 2020-09-03                      1.0
9       Bob 2020-09-04                      3.0
10   Carole 2019-11-11                      1.0
11   Carole 2019-12-25                      1.0
12   Carole 2020-02-01                      1.0
13    David 2019-12-25                      1.0
14      Eva 2019-12-25                      1.0
15      Eva 2019-12-26                      2.0

## Question 4 : Trajets
On veut enrichir les visites avec une information de trajet.
Un trajet représente toutes les visites d'une personne telles que le nombre de jours entre deux visites consécutives soit inférieur à une semaine.

Schema: (photoID, personID, date_num, nb_jours, trajet)

Astuces :
utiliser la réponse à la question 2, la fonction if dans la clause select peut indiquer si la visite est le début du trajet suivant ou non.


In [23]:
%%sql
with T1 as (
  select  photoID , personID, date_num,
    min(date_num) over (
      partition by personId
      order by date_num
      rows between 1 preceding and 1 following
    ) as precedenteDate
  from Visites
  order by personID, date_num),

T2 as (
  select photoID, personID, date_num, datediff('day', precedenteDate, date_num)  as nb_jours
  from T1
  order by personID, date_num),

T3 as (
  select photoID, personID, date_num, nb_jours,
    case when nb_jours >= 7 or nb_jours is null then 1
    else 0
    end as plus1
  from T2
)

select photoID, personID, date_num, nb_jours,
  sum(plus1) over (
    partition by personID
    order by date_num
    rows between unbounded preceding and current row
) as numTrajet
from T3
order by personID, date_num

photoId personId   date_num  nb_jours  numTrajet
0      p16    Alice 2019-12-25         0        0.0
1      p12    Alice 2020-02-01        38        1.0
2      p17    Alice 2020-02-02         1        1.0
3       p2    Alice 2020-09-01       212        2.0
4       p5    Alice 2020-09-04         3        2.0
5       p6    Alice 2020-09-05         1        2.0
6       p7    Alice 2020-10-05        30        3.0
7      p13      Bob 2020-02-01         0        0.0
8       p1      Bob 2020-09-03       215        1.0
9       p3      Bob 2020-09-04         1        1.0
10      p4      Bob 2020-09-04         0        1.0
11     p15   Carole 2019-11-11         0        0.0
12      p8   Carole 2019-12-25        44        1.0
13     p14   Carole 2020-02-01        38        2.0
14      p9    David 2019-12-25         0        0.0
15     p10      Eva 2019-12-25         0        0.0
16     p11      Eva 2019-12-26         1        0.0

### 4 b)
Ajouter la condition: un trajet ne peut pas durer plus de 3 jours au total

In [24]:
#  %%sql


# Données Foursquare

Données issues du réseau social Foursquare

In [25]:
import os
# local_dir = "/local/data"
local_dir = os.environ["HOME"] + "/data"
print("local_dir is", local_dir)
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

local_dir is /root/data


[]

URL pour l'accès aux datasets

In [26]:
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/LqD9N23kxrfHopr"
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL pour les datasets ", PUBLIC_DATASET_URL)

URL pour les datasets  https://nuage.lip6.fr/s/LqD9N23kxrfHopr


In [27]:
import os
from urllib import request
import zipfile

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)

def unzip_file(local_dir, file):
  with zipfile.ZipFile(local_dir + "/" + file, 'r') as zip_ref:
    zip_ref.extractall(local_dir)
  # os.remove(local_dir + "/" + file)


web_dir = PUBLIC_DATASET + "/foursquare"

# ce fichier vient de kaggle : https://www.kaggle.com/datasets/chetanism/foursquare-nyc-and-tokyo-checkin-dataset
zip_filename = "dataset_TSMC2014_NYC.zip"


download_file(web_dir, local_dir, zip_filename)
unzip_file(local_dir, zip_filename)

# Liste des fichiers
os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=/foursquare/dataset_TSMC2014_NYC.zip save in : /root/data/dataset_TSMC2014_NYC.zip


['dataset_TSMC2014_NYC.zip', 'dataset_TSMC2014_NYC.csv']

In [28]:
# !head -4 /local/data/dataset_TSMC2014_NYC.csv
with open(local_dir + "/dataset_TSMC2014_NYC.csv", 'r') as data_file:
    for i in range(4):
        print(data_file.readline().strip())

userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp
470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.71981038,-74.00258103,-240,Tue Apr 03 18:00:09 +0000 2012
979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.60679958,-74.04416981,-240,Tue Apr 03 18:00:25 +0000 2012
69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.71616168,-73.88307006,-240,Tue Apr 03 18:02:24 +0000 2012


## Table Visit

In [29]:
# duckdb.read_csv(f"{local_dir}/notes1M.json")

stmt = f"""
DROP TABLE if exists Visit;
CREATE TABLE Visit as
  SELECT *
  FROM '{local_dir}/dataset_TSMC2014_NYC.csv';
"""
db.sql(stmt)
print("done")

done


### Afficher le contenu d'une table

In [30]:
visit = db.table('Visit')
visit

┌────────┬──────────────────────────┬──────────────────────────┬──────────────────────┬─────────────┬──────────────┬────────────────┬────────────────────────────────┐
│ userId │         venueId          │     venueCategoryId      │    venueCategory     │  latitude   │  longitude   │ timezoneOffset │          utcTimestamp          │
│ int64  │         varchar          │         varchar          │       varchar        │   double    │    double    │     int64      │            varchar             │
├────────┼──────────────────────────┼──────────────────────────┼──────────────────────┼─────────────┼──────────────┼────────────────┼────────────────────────────────┤
│    470 │ 49bbd6c0f964a520f4531fe3 │ 4bf58dd8d48988d127951735 │ Arts & Crafts Store  │ 40.71981038 │ -74.00258103 │           -240 │ Tue Apr 03 18:00:09 +0000 2012 │
│    979 │ 4a43c0aef964a520c6a61fe3 │ 4bf58dd8d48988d1df941735 │ Bridge               │ 40.60679958 │ -74.04416981 │           -240 │ Tue Apr 03 18:00:25 +0000 2012 

In [31]:
display(visit,5)

userId                   venueId           venueCategoryId  \
0     470  49bbd6c0f964a520f4531fe3  4bf58dd8d48988d127951735   
1     979  4a43c0aef964a520c6a61fe3  4bf58dd8d48988d1df941735   
2      69  4c5cc7b485a1e21e00d35711  4bf58dd8d48988d103941735   
3     395  4bc7086715a7ef3bef9878da  4bf58dd8d48988d104941735   
4      87  4cf2c5321d18a143951b5cec  4bf58dd8d48988d1cb941735   

         venueCategory   latitude  longitude  timezoneOffset  \
0  Arts & Crafts Store  40.719810 -74.002581            -240   
1               Bridge  40.606800 -74.044170            -240   
2       Home (private)  40.716162 -73.883070            -240   
3       Medical Center  40.745164 -73.982519            -240   
4           Food Truck  40.740104 -73.989658            -240   

                     utcTimestamp  
0  Tue Apr 03 18:00:09 +0000 2012  
1  Tue Apr 03 18:00:25 +0000 2012  
2  Tue Apr 03 18:02:24 +0000 2012  
3  Tue Apr 03 18:02:41 +0000 2012  
4  Tue Apr 03 18:03:00 +0000 2012

In [32]:
%%sql

select userId, utcTimestamp, count(*) as nb_visites, count(distinct venueId) nb_distinct_poi
from visit
group by userId, utcTimestamp
having nb_visites > 1
order by nb_visites desc
limit 10


userId                    utcTimestamp  nb_visites  nb_distinct_poi
0     739  Thu Apr 19 12:00:12 +0000 2012           4                4
1     739  Sat Apr 21 12:00:10 +0000 2012           4                4
2     739  Thu May 03 12:00:10 +0000 2012           4                4
3     739  Fri May 04 12:00:10 +0000 2012           4                4
4     739  Mon May 14 12:00:11 +0000 2012           4                4
5     344  Mon Nov 19 16:38:14 +0000 2012           4                1
6     739  Sat Apr 28 12:00:10 +0000 2012           4                4
7     739  Tue May 15 12:00:07 +0000 2012           4                4
8     315  Tue Jun 26 23:42:59 +0000 2012           4                1
9     315  Fri Oct 26 05:09:45 +0000 2012           3                1

In [33]:
visit_344_exemple = db.sql("""
select *
from visit
where userId = 344
and utcTimestamp = 'Mon Nov 19 16:38:14 +0000 2012'
""")
display(visit_344_exemple)

userId                   venueId           venueCategoryId  \
0     344  4d51d1dadcce224bb4f9e51b  4bf58dd8d48988d19b941735   
1     344  4d51d1dadcce224bb4f9e51b  4bf58dd8d48988d19b941735   
2     344  4d51d1dadcce224bb4f9e51b  4bf58dd8d48988d19b941735   
3     344  4d51d1dadcce224bb4f9e51b  4bf58dd8d48988d19b941735   

               venueCategory   latitude  longitude  timezoneOffset  \
0  College Academic Building  40.734901  -73.81892            -300   
1  College Academic Building  40.734901  -73.81892            -300   
2  College Academic Building  40.734901  -73.81892            -300   
3  College Academic Building  40.734901  -73.81892            -300   

                     utcTimestamp  
0  Mon Nov 19 16:38:14 +0000 2012  
1  Mon Nov 19 16:38:14 +0000 2012  
2  Mon Nov 19 16:38:14 +0000 2012  
3  Mon Nov 19 16:38:14 +0000 2012

In [34]:
visit_739_exemple = db.sql("""
select *
from visit
where userId = 739
and utcTimestamp = 'Thu Apr 19 12:00:12 +0000 2012'
""")
display(visit_739_exemple)

userId                   venueId           venueCategoryId   venueCategory  \
0     739  4eab92207beb32cb143f9d8d  4bf58dd8d48988d103941735  Home (private)   
1     739  4ba8c654f964a520dbed39e3  4bf58dd8d48988d1fd931735          Subway   
2     739  4f40351fa17cf53abe49a0f4  4bf58dd8d48988d103941735  Home (private)   
3     739  4e8aaccd4fc68f2d71de9384  4bf58dd8d48988d1c7941735     Snack Place   

    latitude  longitude  timezoneOffset                    utcTimestamp  
0  40.818271 -73.906588            -240  Thu Apr 19 12:00:12 +0000 2012  
1  40.816486 -73.907807            -240  Thu Apr 19 12:00:12 +0000 2012  
2  40.818271 -73.906588            -240  Thu Apr 19 12:00:12 +0000 2012  
3  40.815864 -73.907760            -240  Thu Apr 19 12:00:12 +0000 2012

In [35]:
visit_no_duplicate = db.sql("""
 with T1 as (
   select *, row_number() over (partition by userId, utcTimestamp order by utcTimestamp) as event_nb
   from visit
 )
 select  userId, utcTimestamp, venueId, venueCategoryId, venueCategory, latitude, longitude, timezoneOffset
 from T1
 where event_nb = 1
 """)
display(visit_no_duplicate,2)

userId                    utcTimestamp                   venueId  \
0       1  Mon Jul 02 18:16:36 +0000 2012  49d2b43ef964a520cb5b1fe3   
1       1  Mon May 28 21:20:22 +0000 2012  4a9f1cedf964a520903c20e3   

            venueCategoryId       venueCategory   latitude  longitude  \
0  4bf58dd8d48988d1e0931735         Coffee Shop  40.720087 -74.003961   
1  4bf58dd8d48988d1ce941735  Seafood Restaurant  40.788637 -73.974225   

   timezoneOffset  
0            -240  
1            -240

On vérifie qu'il n'y a plus de doublons

In [36]:
db.sql("""
select userId, utcTimestamp, count(*) as nb
from visit_no_duplicate
group by userId, utcTimestamp
having nb > 1
""")

┌────────┬──────────────┬───────┐
│ userId │ utcTimestamp │  nb   │
│ int64  │   varchar    │ int64 │
├────────┴──────────────┴───────┤
│            0 rows             │
└───────────────────────────────┘

### Remplacer la table Visit par celle sans doublons

In [37]:
nb_visites_avant = db.sql("select count(*) from  visit").fetchone()[0]
print("nb de visites avant d'enlever les doublons:", nb_visites_avant)

nb de visites avant d'enlever les doublons: 227428


In [38]:
db.sql("""
create or replace table Visit as
select * from  visit_no_duplicate
""")

In [39]:
nb_visites_apres = db.sql("select count(*) from  visit").fetchone()[0]
print("nb de visites après avoir enlevé les doublons:", nb_visites_apres)

nb de visites après avoir enlevé les doublons: 227102


In [40]:
db.sql("""
select userId, utcTimestamp, count(*) as nb
from visit
group by userId, utcTimestamp
having nb > 1
""")

┌────────┬──────────────┬───────┐
│ userId │ utcTimestamp │  nb   │
│ int64  │   varchar    │ int64 │
├────────┴──────────────┴───────┤
│            0 rows             │
└───────────────────────────────┘

In [41]:
display_schema('Visit')

column_name data_type
0           userId    BIGINT
1     utcTimestamp   VARCHAR
2          venueId   VARCHAR
3  venueCategoryId   VARCHAR
4    venueCategory   VARCHAR
5         latitude    DOUBLE
6        longitude    DOUBLE
7   timezoneOffset    BIGINT

# Requêtes

convertir le timestamp

In [42]:
visit2 = db.sql("""
  SELECT *, strptime(utcTimestamp, '%a %b %d %X %z %Y') as datetime
  FROM visit
""")
display(visit2)

userId                    utcTimestamp                   venueId  \
0        1  Fri Dec 14 17:55:32 +0000 2012  3fd66200f964a52048e31ee3   
1        1  Sat Jun 16 21:29:30 +0000 2012  4cc9a5760f7bef3b5f9c7fdd   
2        1  Sun May 13 00:01:58 +0000 2012  49e69d0cf964a5205b641fe3   
3        1  Sun May 27 21:25:40 +0000 2012  42accc80f964a52047251fe3   
4        1  Thu Jun 07 22:37:34 +0000 2012  42accc80f964a52047251fe3   
5        1  Wed Apr 04 23:31:31 +0000 2012  4abc1f51f964a520798620e3   
6        2  Sat Nov 17 20:52:43 +0000 2012  4a43c0aef964a520c6a61fe3   
7        2  Wed Jun 13 18:01:16 +0000 2012  42829c80f964a5206a221fe3   
8        2  Wed Jun 27 17:23:07 +0000 2012  42ec1480f964a5209e261fe3   
9        3  Fri May 11 13:25:13 +0000 2012  4aa9386ef964a5208b5220e3   
10       3  Fri May 11 23:04:55 +0000 2012  4416912af964a52004311fe3   
11       3  Mon May 28 19:20:46 +0000 2012  4a01ee7bf964a52010711fe3   
12       3  Sat Jun 02 22:59:33 +0000 2012  3fd66200f964a52066ea1ee3   
13       3  Sat Sep 15 16:40:44 +0000 2012  4fcd2197e4b05b50358be24b   
14       3  Tue Apr 10 23:30:58 +0000 2012  440754f8f964a5204d301fe3   
15       3  Wed Jul 18 13:04:14 +0000 2012  4bba2dd2b35776b07b9fca01   
16       4  Fri Nov 16 00:21:24 +0000 2012  4b376908f964a520ef4025e3   
17       4  Mon Dec 31 14:21:13 +0000 2012  4a68fff8f964a5206dcb1fe3   
18       4  Mon Jan 28 23:37:53 +0000 2013  4b376908f964a520ef4025e3   
19       4  Sat Jul 07 00:41:27 +0000 2012  4a3d735cf964a5204ba21fe3   
20       4  Sat May 19 00:03:31 +0000 2012  4b0d3b9df964a520e94423e3   
21       4  Sun Jan 06 17:10:54 +0000 2013  4b376908f964a520ef4025e3   
22       4  Sun Jun 03 12:21:26 +0000 2012  4b376908f964a520ef4025e3   
23       4  Sun Jun 03 23:24:52 +0000 2012  43f5e5b9f964a520a52f1fe3   
24       4  Thu Jan 17 22:26:26 +0000 2013  4ae39d67f964a520489721e3   
25       4  Thu Jul 26 23:42:52 +0000 2012  43fa07c9f964a520d92f1fe3   
26       4  Thu May 03 17:39:39 +0000 2012  4a99749af964a5206a2e20e3   
27       4  Tue Dec 11 23:57:51 +0000 2012  4b376908f964a520ef4025e3   
28       4  Tue May 08 23:42:53 +0000 2012  4b376908f964a520ef4025e3   
29       4  Tue May 15 21:33:53 +0000 2012  4a6101daf964a520abc11fe3   

             venueCategoryId             venueCategory   latitude  longitude  \
0   4bf58dd8d48988d1ce941735        Seafood Restaurant  40.734895 -74.003155   
1   4bf58dd8d48988d1ce941735        Seafood Restaurant  40.745889 -73.988521   
2   4bf58dd8d48988d123941735                       Bar  40.776187 -73.982420   
3   4bf58dd8d48988d123941735                       Bar  40.776624 -73.979482   
4   4bf58dd8d48988d123941735                       Bar  40.776624 -73.979482   
5   4bf58dd8d48988d1ce941735        Seafood Restaurant  40.781558 -73.975792   
6   4bf58dd8d48988d1df941735                    Bridge  40.606800 -74.044170   
7   4bf58dd8d48988d129951735             Train Station  40.752806 -73.977175   
8   4bf58dd8d48988d1e2941735                     Beach  40.567089 -73.865461   
9   4bf58dd8d48988d14e941735       American Restaurant  40.745837 -73.987993   
10  4bf58dd8d48988d11b941735                       Bar  40.751516 -73.993521   
11  4bf58dd8d48988d16c941735              Burger Joint  40.713240 -74.016180   
12  4bf58dd8d48988d14e941735       American Restaurant  40.736982 -74.002197   
13  4bf58dd8d48988d103951735            Clothing Store  40.756870 -73.980997   
14  4bf58dd8d48988d142941735          Asian Restaurant  40.762858 -73.971619   
15  4bf58dd8d48988d124941735                    Office  40.749087 -73.982811   
16  4bf58dd8d48988d132941735                    Church  40.816816 -73.941393   
17  4bf58dd8d48988d176941735      Gym / Fitness Center  40.800361 -73.946811   
18  4bf58dd8d48988d132941735                    Church  40.816816 -73.941393   
19  4bf58dd8d48988d1f6941735          Department Store  40.751502 -73.982719   
20  4bf58dd8d48988d1c0941735  Mediterranean Restaurant  40.804346 -73.947884   
21  4bf58dd

In [43]:
visit3 = db.sql("""
  SELECT userId, venueId, datetime
  FROM visit2
""")

display(visit3)

userId                   venueId                  datetime
0        1  3fd66200f964a52048e31ee3 2012-12-14 17:55:32+00:00
1        1  4cc9a5760f7bef3b5f9c7fdd 2012-06-16 21:29:30+00:00
2        1  49e69d0cf964a5205b641fe3 2012-05-13 00:01:58+00:00
3        1  42accc80f964a52047251fe3 2012-05-27 21:25:40+00:00
4        1  42accc80f964a52047251fe3 2012-06-07 22:37:34+00:00
5        1  4abc1f51f964a520798620e3 2012-04-04 23:31:31+00:00
6        2  4a43c0aef964a520c6a61fe3 2012-11-17 20:52:43+00:00
7        2  42829c80f964a5206a221fe3 2012-06-13 18:01:16+00:00
8        2  42ec1480f964a5209e261fe3 2012-06-27 17:23:07+00:00
9        3  4aa9386ef964a5208b5220e3 2012-05-11 13:25:13+00:00
10       3  4416912af964a52004311fe3 2012-05-11 23:04:55+00:00
11       3  4a01ee7bf964a52010711fe3 2012-05-28 19:20:46+00:00
12       3  3fd66200f964a52066ea1ee3 2012-06-02 22:59:33+00:00
13       3  4fcd2197e4b05b50358be24b 2012-09-15 16:40:44+00:00
14       3  440754f8f964a5204d301fe3 2012-04-10 23:30:58+00:00
15       3  4bba2dd2b35776b07b9fca01 2012-07-18 13:04:14+00:00
16       4  4b376908f964a520ef4025e3 2012-11-16 00:21:24+00:00
17       4  4a68fff8f964a5206dcb1fe3 2012-12-31 14:21:13+00:00
18       4  4b376908f964a520ef4025e3 2013-01-28 23:37:53+00:00
19       4  4a3d735cf964a5204ba21fe3 2012-07-07 00:41:27+00:00
20       4  4b0d3b9df964a520e94423e3 2012-05-19 00:03:31+00:00
21       4  4b376908f964a520ef4025e3 2013-01-06 17:10:54+00:00
22       4  4b376908f964a520ef4025e3 2012-06-03 12:21:26+00:00
23       4  43f5e5b9f964a520a52f1fe3 2012-06-03 23:24:52+00:00
24       4  4ae39d67f964a520489721e3 2013-01-17 22:26:26+00:00
25       4  43fa07c9f964a520d92f1fe3 2012-07-26 23:42:52+00:00
26       4  4a99749af964a5206a2e20e3 2012-05-03 17:39:39+00:00
27       4  4b376908f964a520ef4025e3 2012-12-11 23:57:51+00:00
28       4  4b376908f964a520ef4025e3 2012-05-08 23:42:53+00:00
29       4  4a6101daf964a520abc11fe3 2012-05-15 21:33:53+00:00

##  Ex1 Sequences de visites



### Numéro de POI

Définir les vues
 * venueId_poi(venueId, poi) qui associe le numéro original de venueId avec poi.
   * Indication, utiliser la fonction row_number() over( ...)
 * visit_poi(userId, poi, datetime) avec des numéros de poi allant de 1 à n



In [44]:
venueIds = db.sql("""

  SELECT distinct venueId
  FROM visit2

""")

venueId_poi = db.sql("""

   SELECT venueId, row_number() over (order by venueId) as poi
   FROM venueIds

 """)

display(db.sql("select * from venueId_poi order by poi"))

venueId  poi
0   3fd66200f964a52000e71ee3    1
1   3fd66200f964a52000e81ee3    2
2   3fd66200f964a52000f11ee3    3
3   3fd66200f964a52001e51ee3    4
4   3fd66200f964a52001e81ee3    5
5   3fd66200f964a52002eb1ee3    6
6   3fd66200f964a52003e51ee3    7
7   3fd66200f964a52003e71ee3    8
8   3fd66200f964a52003e81ee3    9
9   3fd66200f964a52004e41ee3   10
10  3fd66200f964a52004e51ee3   11
11  3fd66200f964a52004e61ee3   12
12  3fd66200f964a52004e81ee3   13
13  3fd66200f964a52005e41ee3   14
14  3fd66200f964a52005e51ee3   15
15  3fd66200f964a52005e71ee3   16
16  3fd66200f964a52005e81ee3   17
17  3fd66200f964a52005eb1ee3   18
18  3fd66200f964a52006e61ee3   19
19  3fd66200f964a52006e81ee3   20
20  3fd66200f964a52006e91ee3   21
21  3fd66200f964a52006ea1ee3   22
22  3fd66200f964a52006eb1ee3   23
23  3fd66200f964a52007e61ee3   24
24  3fd66200f964a52007ea1ee3   25
25  3fd66200f964a52007eb1ee3   26
26  3fd66200f964a52008e31ee3   27
27  3fd66200f964a52008e71ee3   28
28  3fd66200f964a52008e81ee3   29
29  3fd66200f964a52008e91ee3   30

In [45]:
db.sql("""
  select count(*)
  from venueId_poi
""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        38333 │
└──────────────┘

In [46]:
visit_poi = db.sql("""
  select v.userId, p.poi, v.datetime
  from visit2 v, venueId_poi p
  where v.venueId = p.venueId
""")

db.sql("select * from visit_poi order by datetime")

┌────────┬───────┬──────────────────────────┐
│ userId │  poi  │         datetime         │
│ int64  │ int64 │ timestamp with time zone │
├────────┼───────┼──────────────────────────┤
│    470 │  2389 │ 2012-04-03 18:00:09+00   │
│    979 │  3922 │ 2012-04-03 18:00:25+00   │
│     69 │ 20329 │ 2012-04-03 18:02:24+00   │
│    395 │ 15115 │ 2012-04-03 18:02:41+00   │
│     87 │ 23551 │ 2012-04-03 18:03:00+00   │
│    484 │ 10590 │ 2012-04-03 18:04:00+00   │
│    642 │  6366 │ 2012-04-03 18:04:38+00   │
│    292 │ 23900 │ 2012-04-03 18:04:42+00   │
│    428 │ 23304 │ 2012-04-03 18:06:18+00   │
│    877 │ 16382 │ 2012-04-03 18:06:19+00   │
│     ·  │   ·   │           ·              │
│     ·  │   ·   │           ·              │
│     ·  │   ·   │           ·              │
│    699 │ 18438 │ 2012-04-11 17:25:52+00   │
│    184 │ 24599 │ 2012-04-11 17:26:52+00   │
│    536 │   936 │ 2012-04-11 17:27:00+00   │
│   1066 │  1793 │ 2012-04-11 17:27:10+00   │
│    974 │  4097 │ 2012-04-11 17:2

### 1.1) Rang
Pour chaque utilisateur, ordonner les visites par date et leur attribuer un **rang** allant de 1 (plus ancienne) à n (la plus récente)

In [47]:
visit_rank = db.sql("""
  SELECT *, rank() over (partition by userId order by datetime) as rank
  from visit_poi
  order by userId, datetime
""")

display(visit_rank)

userId    poi                  datetime  rank
0        1   6428 2012-04-04 23:31:31+00:00     1
1        1  24554 2012-04-07 17:42:24+00:00     2
2        1  26196 2012-04-08 18:20:29+00:00     3
3        1   4131 2012-04-08 20:02:10+00:00     4
4        1    963 2012-04-09 16:20:52+00:00     5
5        1    525 2012-04-10 00:24:31+00:00     6
6        1   1241 2012-04-10 03:36:56+00:00     7
7        1  33237 2012-04-10 16:21:48+00:00     8
8        1   2643 2012-04-12 17:19:21+00:00     9
9        1  33237 2012-04-13 15:41:41+00:00    10
10       1   2150 2012-04-14 01:11:20+00:00    11
11       1  23851 2012-04-14 03:07:56+00:00    12
12       1    983 2012-04-14 04:45:13+00:00    13
13       1   2522 2012-04-14 15:57:20+00:00    14
14       1   1309 2012-04-14 17:45:23+00:00    15
15       1  12421 2012-04-14 20:03:42+00:00    16
16       1   7949 2012-04-14 21:24:08+00:00    17
17       1  30004 2012-04-14 22:07:43+00:00    18
18       1    842 2012-04-14 23:15:07+00:00    19
19       1    842 2012-04-14 23:15:50+00:00    20
20       1   1929 2012-04-16 15:54:00+00:00    21
21       1   7265 2012-04-16 17:56:19+00:00    22
22       1  14858 2012-04-18 16:07:03+00:00    23
23       1  18143 2012-04-19 02:09:21+00:00    24
24       1   2643 2012-04-19 19:02:25+00:00    25
25       1   6070 2012-04-19 22:51:19+00:00    26
26       1    792 2012-04-20 01:49:26+00:00    27
27       1   2076 2012-04-21 00:15:18+00:00    28
28       1  29744 2012-04-21 16:28:11+00:00    29
29       1    817 2012-04-21 18:22:21+00:00    30

### 1.2) Date précédente et déplacement
 On veut connaitre la durée de déplacement séparant deux visites consécutives d'un meme utilisateur.
Pour chaque visite d'un utilisateur, ajouter l'attribut **prev_date** contenant la date de la visite qu'il a effectuée précédemment.

Puis ajouter l'attribut **interval_duration** calculé par différence entre la date courante et la date précédente.
Indication, voir la fonction *date_diff('sec', a, b)*

In [48]:
visit_prev_date = db.sql("""
  select *, min(datetime) over (
    partition by userId
    order by datetime
    rows between 1 preceding and 1 following )  as prev_date
  from visit_rank
  order by userId, rank
""")

display(visit_prev_date)

userId    poi                  datetime  rank                 prev_date
0        1   6428 2012-04-04 23:31:31+00:00     1 2012-04-04 23:31:31+00:00
1        1  24554 2012-04-07 17:42:24+00:00     2 2012-04-04 23:31:31+00:00
2        1  26196 2012-04-08 18:20:29+00:00     3 2012-04-07 17:42:24+00:00
3        1   4131 2012-04-08 20:02:10+00:00     4 2012-04-08 18:20:29+00:00
4        1    963 2012-04-09 16:20:52+00:00     5 2012-04-08 20:02:10+00:00
5        1    525 2012-04-10 00:24:31+00:00     6 2012-04-09 16:20:52+00:00
6        1   1241 2012-04-10 03:36:56+00:00     7 2012-04-10 00:24:31+00:00
7        1  33237 2012-04-10 16:21:48+00:00     8 2012-04-10 03:36:56+00:00
8        1   2643 2012-04-12 17:19:21+00:00     9 2012-04-10 16:21:48+00:00
9        1  33237 2012-04-13 15:41:41+00:00    10 2012-04-12 17:19:21+00:00
10       1   2150 2012-04-14 01:11:20+00:00    11 2012-04-13 15:41:41+00:00
11       1  23851 2012-04-14 03:07:56+00:00    12 2012-04-14 01:11:20+00:00
12       1    983 2012-04-14 04:45:13+00:00    13 2012-04-14 03:07:56+00:00
13       1   2522 2012-04-14 15:57:20+00:00    14 2012-04-14 04:45:13+00:00
14       1   1309 2012-04-14 17:45:23+00:00    15 2012-04-14 15:57:20+00:00
15       1  12421 2012-04-14 20:03:42+00:00    16 2012-04-14 17:45:23+00:00
16       1   7949 2012-04-14 21:24:08+00:00    17 2012-04-14 20:03:42+00:00
17       1  30004 2012-04-14 22:07:43+00:00    18 2012-04-14 21:24:08+00:00
18       1    842 2012-04-14 23:15:07+00:00    19 2012-04-14 22:07:43+00:00
19       1    842 2012-04-14 23:15:50+00:00    20 2012-04-14 23:15:07+00:00
20       1   1929 2012-04-16 15:54:00+00:00    21 2012-04-14 23:15:50+00:00
21       1   7265 2012-04-16 17:56:19+00:00    22 2012-04-16 15:54:00+00:00
22       1  14858 2012-04-18 16:07:03+00:00    23 2012-04-16 17:56:19+00:00
23       1  18143 2012-04-19 02:09:21+00:00    24 2012-04-18 16:07:03+00:00
24       1   2643 2012-04-19 19:02:25+00:00    25 2012-04-19 02:09:21+00:00
25       1   6070 2012-04-19 22:51:19+00:00    26 2012-04-19 19:02:25+00:00
26       1    792 2012-04-20 01:49:26+00:00    27 2012-04-19 22:51:19+00:00
27       1   2076 2012-04-21 00:15:18+00:00    28 2012-04-20 01:49:26+00:00
28       1  29744 2012-04-21 16:28:11+00:00    29 2012-04-21 00:15:18+00:00
29       1    817 2012-04-21 18:22:21+00:00    30 2012-04-21 16:28:11+00:00

In [49]:
visit_duration = db.sql("""
  select *, datediff('sec', prev_date, datetime) as interval_duration
  from visit_prev_date
  order by userId, rank
""")

display(visit_duration)

userId    poi                  datetime  rank                 prev_date  \
0        1   6428 2012-04-04 23:31:31+00:00     1 2012-04-04 23:31:31+00:00   
1        1  24554 2012-04-07 17:42:24+00:00     2 2012-04-04 23:31:31+00:00   
2        1  26196 2012-04-08 18:20:29+00:00     3 2012-04-07 17:42:24+00:00   
3        1   4131 2012-04-08 20:02:10+00:00     4 2012-04-08 18:20:29+00:00   
4        1    963 2012-04-09 16:20:52+00:00     5 2012-04-08 20:02:10+00:00   
5        1    525 2012-04-10 00:24:31+00:00     6 2012-04-09 16:20:52+00:00   
6        1   1241 2012-04-10 03:36:56+00:00     7 2012-04-10 00:24:31+00:00   
7        1  33237 2012-04-10 16:21:48+00:00     8 2012-04-10 03:36:56+00:00   
8        1   2643 2012-04-12 17:19:21+00:00     9 2012-04-10 16:21:48+00:00   
9        1  33237 2012-04-13 15:41:41+00:00    10 2012-04-12 17:19:21+00:00   
10       1   2150 2012-04-14 01:11:20+00:00    11 2012-04-13 15:41:41+00:00   
11       1  23851 2012-04-14 03:07:56+00:00    12 2012-04-14 01:11:20+00:00   
12       1    983 2012-04-14 04:45:13+00:00    13 2012-04-14 03:07:56+00:00   
13       1   2522 2012-04-14 15:57:20+00:00    14 2012-04-14 04:45:13+00:00   
14       1   1309 2012-04-14 17:45:23+00:00    15 2012-04-14 15:57:20+00:00   
15       1  12421 2012-04-14 20:03:42+00:00    16 2012-04-14 17:45:23+00:00   
16       1   7949 2012-04-14 21:24:08+00:00    17 2012-04-14 20:03:42+00:00   
17       1  30004 2012-04-14 22:07:43+00:00    18 2012-04-14 21:24:08+00:00   
18       1    842 2012-04-14 23:15:07+00:00    19 2012-04-14 22:07:43+00:00   
19       1    842 2012-04-14 23:15:50+00:00    20 2012-04-14 23:15:07+00:00   
20       1   1929 2012-04-16 15:54:00+00:00    21 2012-04-14 23:15:50+00:00   
21       1   7265 2012-04-16 17:56:19+00:00    22 2012-04-16 15:54:00+00:00   
22       1  14858 2012-04-18 16:07:03+00:00    23 2012-04-16 17:56:19+00:00   
23       1  18143 2012-04-19 02:09:21+00:00    24 2012-04-18 16:07:03+00:00   
24       1   2643 2012-04-19 19:02:25+00:00    25 2012-04-19 02:09:21+00:00   
25       1   6070 2012-04-19 22:51:19+00:00    26 2012-04-19 19:02:25+00:00   
26       1    792 2012-04-20 01:49:26+00:00    27 2012-04-19 22:51:19+00:00   
27       1   2076 2012-04-21 00:15:18+00:00    28 2012-04-20 01:49:26+00:00   
28       1  29744 2012-04-21 16:28:11+00:00    29 2012-04-21 00:15:18+00:00   
29       1    817 2012-04-21 18:22:21+00:00    30 2012-04-21 16:28:11+00:00   

    interval_duration  
0                   0  
1              238253  
2               88685  
3                6101  
4               73122  
5               29019  
6               11545  
7               45892  
8              176253  
9               80540  
10              34179  
11               6996  
12               5837  
13              40327  
14               6483  
15               8299  
16               4826  
17               2615  
18               4044  
19                 43  
20             146290  
21               7339  
22             166244  
23              36138  
24              60784  
25              13734  
26              10687  
27              80752  
28              58373  
29               6850

### 1.3) Numéro de trajectoire

Séparer les visites de chaque utilisateur en trajectoires.

  * Dans une trajectoire, la durée entre deux visites consécutives ne peut pas dépasser 20 heures (soit 20 * 3600 secondes).

Ajouter l'attribut **traj** indiquant le numéro de la trajectoire pour un utilisateur donné.
Indication: on peut utiliser une expression *case when then else end*

In [50]:
visit_traj1 = db.sql("""
select *,
  case when interval_duration > 20*3600 or interval_duration = 0 then 1
  else 0
  end as debut
from visit_duration
""")


display(visit_traj1)

userId    poi                  datetime  rank                 prev_date  \
0        1   6428 2012-04-04 23:31:31+00:00     1 2012-04-04 23:31:31+00:00   
1        1  24554 2012-04-07 17:42:24+00:00     2 2012-04-04 23:31:31+00:00   
2        1  26196 2012-04-08 18:20:29+00:00     3 2012-04-07 17:42:24+00:00   
3        1   4131 2012-04-08 20:02:10+00:00     4 2012-04-08 18:20:29+00:00   
4        1    963 2012-04-09 16:20:52+00:00     5 2012-04-08 20:02:10+00:00   
5        1    525 2012-04-10 00:24:31+00:00     6 2012-04-09 16:20:52+00:00   
6        1   1241 2012-04-10 03:36:56+00:00     7 2012-04-10 00:24:31+00:00   
7        1  33237 2012-04-10 16:21:48+00:00     8 2012-04-10 03:36:56+00:00   
8        1   2643 2012-04-12 17:19:21+00:00     9 2012-04-10 16:21:48+00:00   
9        1  33237 2012-04-13 15:41:41+00:00    10 2012-04-12 17:19:21+00:00   
10       1   2150 2012-04-14 01:11:20+00:00    11 2012-04-13 15:41:41+00:00   
11       1  23851 2012-04-14 03:07:56+00:00    12 2012-04-14 01:11:20+00:00   
12       1    983 2012-04-14 04:45:13+00:00    13 2012-04-14 03:07:56+00:00   
13       1   2522 2012-04-14 15:57:20+00:00    14 2012-04-14 04:45:13+00:00   
14       1   1309 2012-04-14 17:45:23+00:00    15 2012-04-14 15:57:20+00:00   
15       1  12421 2012-04-14 20:03:42+00:00    16 2012-04-14 17:45:23+00:00   
16       1   7949 2012-04-14 21:24:08+00:00    17 2012-04-14 20:03:42+00:00   
17       1  30004 2012-04-14 22:07:43+00:00    18 2012-04-14 21:24:08+00:00   
18       1    842 2012-04-14 23:15:07+00:00    19 2012-04-14 22:07:43+00:00   
19       1    842 2012-04-14 23:15:50+00:00    20 2012-04-14 23:15:07+00:00   
20       1   1929 2012-04-16 15:54:00+00:00    21 2012-04-14 23:15:50+00:00   
21       1   7265 2012-04-16 17:56:19+00:00    22 2012-04-16 15:54:00+00:00   
22       1  14858 2012-04-18 16:07:03+00:00    23 2012-04-16 17:56:19+00:00   
23       1  18143 2012-04-19 02:09:21+00:00    24 2012-04-18 16:07:03+00:00   
24       1   2643 2012-04-19 19:02:25+00:00    25 2012-04-19 02:09:21+00:00   
25       1   6070 2012-04-19 22:51:19+00:00    26 2012-04-19 19:02:25+00:00   
26       1    792 2012-04-20 01:49:26+00:00    27 2012-04-19 22:51:19+00:00   
27       1   2076 2012-04-21 00:15:18+00:00    28 2012-04-20 01:49:26+00:00   
28       1  29744 2012-04-21 16:28:11+00:00    29 2012-04-21 00:15:18+00:00   
29       1    817 2012-04-21 18:22:21+00:00    30 2012-04-21 16:28:11+00:00   

    interval_duration  debut  
0                   0      1  
1              238253      1  
2               88685      1  
3                6101      0  
4               73122      1  
5               29019      0  
6               11545      0  
7               45892      0  
8              176253      1  
9               80540      1  
10              34179      0  
11               6996      0  
12               5837      0  
13              40327      0  
14               6483      0  
15               8299      0  
16               4826      0  
17               2615      0  
18               4044      0  
19                 43      0  
20             146290      1  
21               7339      0  
22             166244      1  
23              36138      0  
24              60784      0  
25              13734      0  
26              10687      0  
27              80752      1  
28              58373      0  
29               6850      0

In [51]:
visit_traj2 = db.sql("""
select *, sum(debut) over (
  partition by userId
  order by datetime
) as traj
from visit_traj1
order by userId, rank

""")

display(visit_traj2)

userId    poi                  datetime  rank                 prev_date  \
0        1   6428 2012-04-04 23:31:31+00:00     1 2012-04-04 23:31:31+00:00   
1        1  24554 2012-04-07 17:42:24+00:00     2 2012-04-04 23:31:31+00:00   
2        1  26196 2012-04-08 18:20:29+00:00     3 2012-04-07 17:42:24+00:00   
3        1   4131 2012-04-08 20:02:10+00:00     4 2012-04-08 18:20:29+00:00   
4        1    963 2012-04-09 16:20:52+00:00     5 2012-04-08 20:02:10+00:00   
5        1    525 2012-04-10 00:24:31+00:00     6 2012-04-09 16:20:52+00:00   
6        1   1241 2012-04-10 03:36:56+00:00     7 2012-04-10 00:24:31+00:00   
7        1  33237 2012-04-10 16:21:48+00:00     8 2012-04-10 03:36:56+00:00   
8        1   2643 2012-04-12 17:19:21+00:00     9 2012-04-10 16:21:48+00:00   
9        1  33237 2012-04-13 15:41:41+00:00    10 2012-04-12 17:19:21+00:00   
10       1   2150 2012-04-14 01:11:20+00:00    11 2012-04-13 15:41:41+00:00   
11       1  23851 2012-04-14 03:07:56+00:00    12 2012-04-14 01:11:20+00:00   
12       1    983 2012-04-14 04:45:13+00:00    13 2012-04-14 03:07:56+00:00   
13       1   2522 2012-04-14 15:57:20+00:00    14 2012-04-14 04:45:13+00:00   
14       1   1309 2012-04-14 17:45:23+00:00    15 2012-04-14 15:57:20+00:00   
15       1  12421 2012-04-14 20:03:42+00:00    16 2012-04-14 17:45:23+00:00   
16       1   7949 2012-04-14 21:24:08+00:00    17 2012-04-14 20:03:42+00:00   
17       1  30004 2012-04-14 22:07:43+00:00    18 2012-04-14 21:24:08+00:00   
18       1    842 2012-04-14 23:15:07+00:00    19 2012-04-14 22:07:43+00:00   
19       1    842 2012-04-14 23:15:50+00:00    20 2012-04-14 23:15:07+00:00   
20       1   1929 2012-04-16 15:54:00+00:00    21 2012-04-14 23:15:50+00:00   
21       1   7265 2012-04-16 17:56:19+00:00    22 2012-04-16 15:54:00+00:00   
22       1  14858 2012-04-18 16:07:03+00:00    23 2012-04-16 17:56:19+00:00   
23       1  18143 2012-04-19 02:09:21+00:00    24 2012-04-18 16:07:03+00:00   
24       1   2643 2012-04-19 19:02:25+00:00    25 2012-04-19 02:09:21+00:00   
25       1   6070 2012-04-19 22:51:19+00:00    26 2012-04-19 19:02:25+00:00   
26       1    792 2012-04-20 01:49:26+00:00    27 2012-04-19 22:51:19+00:00   
27       1   2076 2012-04-21 00:15:18+00:00    28 2012-04-20 01:49:26+00:00   
28       1  29744 2012-04-21 16:28:11+00:00    29 2012-04-21 00:15:18+00:00   
29       1    817 2012-04-21 18:22:21+00:00    30 2012-04-21 16:28:11+00:00   

    interval_duration  debut  traj  
0                   0      1   1.0  
1              238253      1   2.0  
2               88685      1   3.0  
3                6101      0   3.0  
4               73122      1   4.0  
5               29019      0   4.0  
6               11545      0   4.0  
7               45892      0   4.0  
8              176253      1   5.0  
9               80540      1   6.0  
10              34179      0   6.0  
11               6996      0   6.0  
12               5837      0   6.0  
13              40327      0   6.0  
14               6483      0   6.0  
15               8299      0   6.0  
16               4826      0   6.0  
17               2615      0   6.0  
18               4044      0   6.0  
19                 43      0   6.0  
20             146290      1   7.0  
21               7339      0   7.0  
22             166244      1   8.0  
23              36138      0   8.0  
24              60784      0   8.0  
25              13734      0   8.0  
26              10687      0   8.0  
27              80752      1   9.0  
28              58373      0   9.0  
29               6850      0   9.0

## Ex2: Fenètres glissantes

### 2.1) Nombre de POI visités en une semaine

Ajouter l'attribut **nb_POI_7jours** donnant le nombre de POI visités dans les 7 jours qui précèdent le jour de la visite courante. Ne pas

In [52]:
# pour ne pas inclure les visites du jour courant, la borne supérieure de la fenetre precede de 1 jour la date courante
visit_7j = db.sql("""
select * , count(*) over (
  partition by userId
  order by datetime
  range between interval 6 day preceding and interval 0 day following
) as nb_POI_7jours
from visit_poi
order by userid, datetime
""")
display(visit_7j)

userId    poi                  datetime  nb_POI_7jours
0        1   6428 2012-04-04 23:31:31+00:00              1
1        1  24554 2012-04-07 17:42:24+00:00              2
2        1  26196 2012-04-08 18:20:29+00:00              3
3        1   4131 2012-04-08 20:02:10+00:00              4
4        1    963 2012-04-09 16:20:52+00:00              5
5        1    525 2012-04-10 00:24:31+00:00              6
6        1   1241 2012-04-10 03:36:56+00:00              7
7        1  33237 2012-04-10 16:21:48+00:00              8
8        1   2643 2012-04-12 17:19:21+00:00              8
9        1  33237 2012-04-13 15:41:41+00:00              9
10       1   2150 2012-04-14 01:11:20+00:00              9
11       1  23851 2012-04-14 03:07:56+00:00             10
12       1    983 2012-04-14 04:45:13+00:00             11
13       1   2522 2012-04-14 15:57:20+00:00             12
14       1   1309 2012-04-14 17:45:23+00:00             13
15       1  12421 2012-04-14 20:03:42+00:00             12
16       1   7949 2012-04-14 21:24:08+00:00             13
17       1  30004 2012-04-14 22:07:43+00:00             14
18       1    842 2012-04-14 23:15:07+00:00             15
19       1    842 2012-04-14 23:15:50+00:00             16
20       1   1929 2012-04-16 15:54:00+00:00             14
21       1   7265 2012-04-16 17:56:19+00:00             14
22       1  14858 2012-04-18 16:07:03+00:00             15
23       1  18143 2012-04-19 02:09:21+00:00             15
24       1   2643 2012-04-19 19:02:25+00:00             15
25       1   6070 2012-04-19 22:51:19+00:00             16
26       1    792 2012-04-20 01:49:26+00:00             16
27       1   2076 2012-04-21 00:15:18+00:00              8
28       1  29744 2012-04-21 16:28:11+00:00              9
29       1    817 2012-04-21 18:22:21+00:00             10

### 2.2) Nombre cumulé de visites par utilisateur

Ajouter un attribut **cumul_visites** indiquant le nombre de POI qu'un utilisateur a déjà visités auparavant.

Indications: Le cumul n'est pas agrégé par utilisateur. Pour chaque visite, déterminer combien de POI ont déja été visités par cet utilisateur avant la visite courante.


In [53]:
# # Tenir compte dans le cumul des POI visités plusieurs fois : les compter qu'une seule fois
visit_cumul = db.sql("""
select userId, poi, datetime, count(*) over (
  partition by userId
  order by datetime
  rows between unbounded preceding and 0 preceding
) as cumul_visit
from visit_7j
order by userId, datetime
""")

db.sql("select *, from visit_cumul")

┌────────┬───────┬──────────────────────────┬─────────────┐
│ userId │  poi  │         datetime         │ cumul_visit │
│ int64  │ int64 │ timestamp with time zone │    int64    │
├────────┼───────┼──────────────────────────┼─────────────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │           1 │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │           2 │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │           3 │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │           4 │
│      1 │   963 │ 2012-04-09 16:20:52+00   │           5 │
│      1 │   525 │ 2012-04-10 00:24:31+00   │           6 │
│      1 │  1241 │ 2012-04-10 03:36:56+00   │           7 │
│      1 │ 33237 │ 2012-04-10 16:21:48+00   │           8 │
│      1 │  2643 │ 2012-04-12 17:19:21+00   │           9 │
│      1 │ 33237 │ 2012-04-13 15:41:41+00   │          10 │
│      · │    ·  │           ·              │           · │
│      · │    ·  │           ·              │           · │
│      · │    ·  │           ·          

## Exercice 3

### 3.1) Visites dans une trajectoire
On constate qu'une trajectoire peut contenir plusieurs visites consécutives d'un même POI et on veut "fusionner" ces visites.
Pour chaque trajectoire, agréger les visites consécutives d'un meme POI. Définir les attributs date_début, date_fin pour la visite d'un POI.

Rmq1: s'il n'y a pas plusieurs visites consécutives pour un POI alors ses dates de début et de fin sont identiques.

Rmq2: une trajectoire peut contenir plusieurs visites non consécutives d'un même POI.

Autre solution avec first() au lieu de lag()

In [54]:
trajectoire1 = db.sql("""
select userId, poi, datetime, rank, interval_duration, traj,
     case when lag(poi) over (partition by userId, traj order by rank) is null
     then poi
     else lag(poi) over (partition by userId, traj order by rank)
     end as prev_poi
from visit_traj2
order by userId, traj, rank

""")
display(trajectoire1)

userId    poi                  datetime  rank  interval_duration  traj  \
0        1   6428 2012-04-04 23:31:31+00:00     1                  0   1.0   
1        1  24554 2012-04-07 17:42:24+00:00     2             238253   2.0   
2        1  26196 2012-04-08 18:20:29+00:00     3              88685   3.0   
3        1   4131 2012-04-08 20:02:10+00:00     4               6101   3.0   
4        1    963 2012-04-09 16:20:52+00:00     5              73122   4.0   
5        1    525 2012-04-10 00:24:31+00:00     6              29019   4.0   
6        1   1241 2012-04-10 03:36:56+00:00     7              11545   4.0   
7        1  33237 2012-04-10 16:21:48+00:00     8              45892   4.0   
8        1   2643 2012-04-12 17:19:21+00:00     9             176253   5.0   
9        1  33237 2012-04-13 15:41:41+00:00    10              80540   6.0   
10       1   2150 2012-04-14 01:11:20+00:00    11              34179   6.0   
11       1  23851 2012-04-14 03:07:56+00:00    12               6996   6.0   
12       1    983 2012-04-14 04:45:13+00:00    13               5837   6.0   
13       1   2522 2012-04-14 15:57:20+00:00    14              40327   6.0   
14       1   1309 2012-04-14 17:45:23+00:00    15               6483   6.0   
15       1  12421 2012-04-14 20:03:42+00:00    16               8299   6.0   
16       1   7949 2012-04-14 21:24:08+00:00    17               4826   6.0   
17       1  30004 2012-04-14 22:07:43+00:00    18               2615   6.0   
18       1    842 2012-04-14 23:15:07+00:00    19               4044   6.0   
19       1    842 2012-04-14 23:15:50+00:00    20                 43   6.0   
20       1   1929 2012-04-16 15:54:00+00:00    21             146290   7.0   
21       1   7265 2012-04-16 17:56:19+00:00    22               7339   7.0   
22       1  14858 2012-04-18 16:07:03+00:00    23             166244   8.0   
23       1  18143 2012-04-19 02:09:21+00:00    24              36138   8.0   
24       1   2643 2012-04-19 19:02:25+00:00    25              60784   8.0   
25       1   6070 2012-04-19 22:51:19+00:00    26              13734   8.0   
26       1    792 2012-04-20 01:49:26+00:00    27              10687   8.0   
27       1   2076 2012-04-21 00:15:18+00:00    28              80752   9.0   
28       1  29744 2012-04-21 16:28:11+00:00    29              58373   9.0   
29       1    817 2012-04-21 18:22:21+00:00    30               6850   9.0   

    prev_poi  
0       6428  
1      24554  
2      26196  
3      26196  
4        963  
5        963  
6        525  
7       1241  
8       2643  
9      33237  
10     33237  
11      2150  
12     23851  
13       983  
14      2522  
15      1309  
16     12421  
17      7949  
18     30004  
19       842  
20      1929  
21      1929  
22     14858  
23     14858  
24     18143  
25      2643  
26      6070  
27      2076  
28      2076  
29     29744

In [55]:
trajectoire2 = db.sql("""
select *, rank() over (partition by userId, traj order by rank) as  rank_in_traj
from trajectoire1
order by userId, traj, rank_in_traj
""")

display(trajectoire2)

userId    poi                  datetime  rank  interval_duration  traj  \
0        1   6428 2012-04-04 23:31:31+00:00     1                  0   1.0   
1        1  24554 2012-04-07 17:42:24+00:00     2             238253   2.0   
2        1  26196 2012-04-08 18:20:29+00:00     3              88685   3.0   
3        1   4131 2012-04-08 20:02:10+00:00     4               6101   3.0   
4        1    963 2012-04-09 16:20:52+00:00     5              73122   4.0   
5        1    525 2012-04-10 00:24:31+00:00     6              29019   4.0   
6        1   1241 2012-04-10 03:36:56+00:00     7              11545   4.0   
7        1  33237 2012-04-10 16:21:48+00:00     8              45892   4.0   
8        1   2643 2012-04-12 17:19:21+00:00     9             176253   5.0   
9        1  33237 2012-04-13 15:41:41+00:00    10              80540   6.0   
10       1   2150 2012-04-14 01:11:20+00:00    11              34179   6.0   
11       1  23851 2012-04-14 03:07:56+00:00    12               6996   6.0   
12       1    983 2012-04-14 04:45:13+00:00    13               5837   6.0   
13       1   2522 2012-04-14 15:57:20+00:00    14              40327   6.0   
14       1   1309 2012-04-14 17:45:23+00:00    15               6483   6.0   
15       1  12421 2012-04-14 20:03:42+00:00    16               8299   6.0   
16       1   7949 2012-04-14 21:24:08+00:00    17               4826   6.0   
17       1  30004 2012-04-14 22:07:43+00:00    18               2615   6.0   
18       1    842 2012-04-14 23:15:07+00:00    19               4044   6.0   
19       1    842 2012-04-14 23:15:50+00:00    20                 43   6.0   
20       1   1929 2012-04-16 15:54:00+00:00    21             146290   7.0   
21       1   7265 2012-04-16 17:56:19+00:00    22               7339   7.0   
22       1  14858 2012-04-18 16:07:03+00:00    23             166244   8.0   
23       1  18143 2012-04-19 02:09:21+00:00    24              36138   8.0   
24       1   2643 2012-04-19 19:02:25+00:00    25              60784   8.0   
25       1   6070 2012-04-19 22:51:19+00:00    26              13734   8.0   
26       1    792 2012-04-20 01:49:26+00:00    27              10687   8.0   
27       1   2076 2012-04-21 00:15:18+00:00    28              80752   9.0   
28       1  29744 2012-04-21 16:28:11+00:00    29              58373   9.0   
29       1    817 2012-04-21 18:22:21+00:00    30               6850   9.0   

    prev_poi  rank_in_traj  
0       6428             1  
1      24554             1  
2      26196             1  
3      26196             2  
4        963             1  
5        963             2  
6        525             3  
7       1241             4  
8       2643             1  
9      33237             1  
10     33237             2  
11      2150             3  
12     23851             4  
13       983             5  
14      2522             6  
15      1309             7  
16     12421             8  
17      7949             9  
18     30004            10  
19       842            11  
20      1929             1  
21      1929             2  
22     14858             1  
23     14858             2  
24     18143             3  
25      2643             4  
26      6070             5  
27      2076             1  
28      2076             2  
29     29744             3

In [56]:
trajectoire3 = db.sql("""
select *,
  case when lag(poi) over (partition by userId, traj order by rank) = poi then 0
  else 1
  end  as debut_poi
from trajectoire2
order by userId, traj, rank

""")

trajectoire3

┌────────┬───────┬──────────────────────────┬───────┬───────────────────┬────────┬──────────┬──────────────┬───────────┐
│ userId │  poi  │         datetime         │ rank  │ interval_duration │  traj  │ prev_poi │ rank_in_traj │ debut_poi │
│ int64  │ int64 │ timestamp with time zone │ int64 │       int64       │ int128 │  int64   │    int64     │   int32   │
├────────┼───────┼──────────────────────────┼───────┼───────────────────┼────────┼──────────┼──────────────┼───────────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │                 0 │      1 │     6428 │            1 │         1 │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │            238253 │      2 │    24554 │            1 │         1 │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │             88685 │      3 │    26196 │            1 │         1 │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │              6101 │      3 │    26196 │            2 │         1 │
│      1 │   963 │ 2012-04-09 16

afficher les visites de l'utilisateur 984

In [57]:
db.sql("""
select *
from trajectoire3
where userId = 984
order by traj
""")

┌────────┬───────┬──────────────────────────┬───────┬───────────────────┬────────┬──────────┬──────────────┬───────────┐
│ userId │  poi  │         datetime         │ rank  │ interval_duration │  traj  │ prev_poi │ rank_in_traj │ debut_poi │
│ int64  │ int64 │ timestamp with time zone │ int64 │       int64       │ int128 │  int64   │    int64     │   int32   │
├────────┼───────┼──────────────────────────┼───────┼───────────────────┼────────┼──────────┼──────────────┼───────────┤
│    984 │  2532 │ 2012-04-03 21:31:50+00   │     1 │                 0 │      1 │     2532 │            1 │         1 │
│    984 │  2532 │ 2012-04-04 09:30:00+00   │     2 │             43090 │      1 │     2532 │            2 │         0 │
│    984 │  2532 │ 2012-04-05 10:06:32+00   │     3 │             88592 │      2 │     2532 │            1 │         1 │
│    984 │  2532 │ 2012-04-10 22:40:41+00   │     4 │            477249 │      3 │     2532 │            1 │         1 │
│    984 │  2532 │ 2012-04-11 09

In [62]:
trajectoire4 = db.sql("""
select *, rank_in_traj  as visitId
from trajectoire3
""")

trajectoire4

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────┬───────┬──────────────────────────┬───────┬───────────────────┬────────┬──────────┬──────────────┬───────────┬─────────┐
│ userId │  poi  │         datetime         │ rank  │ interval_duration │  traj  │ prev_poi │ rank_in_traj │ debut_poi │ visitId │
│ int64  │ int64 │ timestamp with time zone │ int64 │       int64       │ int128 │  int64   │    int64     │   int32   │  int64  │
├────────┼───────┼──────────────────────────┼───────┼───────────────────┼────────┼──────────┼──────────────┼───────────┼─────────┤
│      1 │  6428 │ 2012-04-04 23:31:31+00   │     1 │                 0 │      1 │     6428 │            1 │         1 │       1 │
│      1 │ 24554 │ 2012-04-07 17:42:24+00   │     2 │            238253 │      2 │    24554 │            1 │         1 │       1 │
│      1 │ 26196 │ 2012-04-08 18:20:29+00   │     3 │             88685 │      3 │    26196 │            1 │         1 │       1 │
│      1 │  4131 │ 2012-04-08 20:02:10+00   │     4 │              6101 │      3 │ 

In [66]:
cube1 = db.sql("""
select userId, traj, poi, min(datetime) as date_in, max(datetime) as date_out,
  count(*) as nb_checkins
from trajectoire4
group by cube (userId, traj, poi)
order by userId, traj, date_in
""")

trajectoire5 = db.sql("""
select *, rank() over (partition by userId, traj order by date_in) as visitId
from cube1
where userId is not null and traj is not null and poi is not null
order by userId, traj, date_in
""")

trajectoire5

┌────────┬────────┬───────┬──────────────────────────┬──────────────────────────┬─────────────┬─────────┐
│ userId │  traj  │  poi  │         date_in          │         date_out         │ nb_checkins │ visitId │
│ int64  │ int128 │ int64 │ timestamp with time zone │ timestamp with time zone │    int64    │  int64  │
├────────┼────────┼───────┼──────────────────────────┼──────────────────────────┼─────────────┼─────────┤
│      1 │      1 │  6428 │ 2012-04-04 23:31:31+00   │ 2012-04-04 23:31:31+00   │           1 │       1 │
│      1 │      2 │ 24554 │ 2012-04-07 17:42:24+00   │ 2012-04-07 17:42:24+00   │           1 │       1 │
│      1 │      3 │ 26196 │ 2012-04-08 18:20:29+00   │ 2012-04-08 18:20:29+00   │           1 │       1 │
│      1 │      3 │  4131 │ 2012-04-08 20:02:10+00   │ 2012-04-08 20:02:10+00   │           1 │       2 │
│      1 │      4 │   963 │ 2012-04-09 16:20:52+00   │ 2012-04-09 16:20:52+00   │           1 │       1 │
│      1 │      4 │   525 │ 2012-04-10 00:24:3

In [67]:
trajectoire6 = db.sql("""
select userId, traj, visitId, poi, nb_checkins, date_in, date_out, datediff('sec', date_in, date_out) as visit_duration
from trajectoire5_bis
order by userId, traj, date_in
""")

trajectoire6

┌────────┬────────┬─────────┬───────┬─────────────┬──────────────────────────┬──────────────────────────┬────────────────┐
│ userId │  traj  │ visitId │  poi  │ nb_checkins │         date_in          │         date_out         │ visit_duration │
│ int64  │ int128 │  int64  │ int64 │    int64    │ timestamp with time zone │ timestamp with time zone │     int64      │
├────────┼────────┼─────────┼───────┼─────────────┼──────────────────────────┼──────────────────────────┼────────────────┤
│      1 │      1 │       1 │  6428 │           1 │ 2012-04-04 23:31:31+00   │ 2012-04-04 23:31:31+00   │              0 │
│      1 │      2 │       1 │ 24554 │           1 │ 2012-04-07 17:42:24+00   │ 2012-04-07 17:42:24+00   │              0 │
│      1 │      3 │       1 │ 26196 │           1 │ 2012-04-08 18:20:29+00   │ 2012-04-08 18:20:29+00   │              0 │
│      1 │      3 │       2 │  4131 │           1 │ 2012-04-08 20:02:10+00   │ 2012-04-08 20:02:10+00   │              0 │
│      1 │      

In [68]:
db.sql("""
select *
from trajectoire6
where userId=984
order by traj
""")

┌────────┬────────┬─────────┬───────┬─────────────┬──────────────────────────┬──────────────────────────┬────────────────┐
│ userId │  traj  │ visitId │  poi  │ nb_checkins │         date_in          │         date_out         │ visit_duration │
│ int64  │ int128 │  int64  │ int64 │    int64    │ timestamp with time zone │ timestamp with time zone │     int64      │
├────────┼────────┼─────────┼───────┼─────────────┼──────────────────────────┼──────────────────────────┼────────────────┤
│    984 │      1 │       1 │  2532 │           2 │ 2012-04-03 21:31:50+00   │ 2012-04-04 09:30:00+00   │          43090 │
│    984 │      2 │       1 │  2532 │           1 │ 2012-04-05 10:06:32+00   │ 2012-04-05 10:06:32+00   │              0 │
│    984 │      3 │       1 │  2532 │           2 │ 2012-04-10 22:40:41+00   │ 2012-04-11 09:30:15+00   │          38974 │
│    984 │      3 │       2 │ 20194 │           1 │ 2012-04-11 16:03:05+00   │ 2012-04-11 16:03:05+00   │              0 │
│    984 │      